# Deep Residual Learning for Image Recognition Review

ResNet에 관한 논문을 리뷰하겠습니다. 전반적인 흐름은 논문 순서대로 진행되며, 나름 요약해서 재구성한 내용입니다. 반복되어 언급되는 성과나 수상들은 웬만하면 빼겠습니다.

## Abstract

   __딥러닝이 깊어지면 깊어질수록 학습은 어려워집니다.__ 이에 대한 해결책으로 이 논문에서는 __residual learning framework__를 제안합니다. 이 프레임워크를 사용하면 이전의 연구에서 사용했던 네트워크들보다 __더 깊으면서도 학습이 용이한 네트워크__를 가능케 합니다.  
  
   여기서 제안하는 방법은 지금까지 우리가 알아왔던 일반적인 layer 방식이 아닌, __layer의 input을 참고하는 residual function을 학습하게끔 layer를 재구성합니다.__ 이 방식의 네트워크(residual networks)는 더 쉽게 최적화할 수 있고, 증가한 depth에서 더 좋은 accuracy를 얻을 수 있습니다.

## Introduction

현재 Deep CNN은 이미지 인식 분야에서 굉장히 중요한 역할을 하고 있습니다. 특히 최근 challenging ImageNet dataset에서 16 ~ 30층 정도의 depth가 결과를 얻으면서 네트워크의 depth가 매우 중요해졌습니다.  
  
이에 논문에서는 이런 의문이 들었습니다. __"layer를 많이 쌓으면 쌓을수록 network의 학습은 더 나아지지 않을까?"__  
  
이 의문을 풀어가는데 처음부터 문제가 발생하는데, 바로 vanishing/exploding gradients 문제입니다. 그러나 이 문제는 normalized initialization과 intermediate normalization으로 해결해왔습니다.  
  
이 해결책들을 바탕으로 deep network를 학습시키면, degradation problem이 나타납니다. __degradation problem이란 network의 depth가 깊어지면 accuracy가 하락하는 현상입니다.__ 그러나 이는 overfitting에 의한 것이 아닙니다. depth가 적절한 모델에 추가적인 layer를 쌓게 되면, training error가 증가하는 것입니다. 아래의 그림을 보면 더 확실히 알 수 있습니다. overfitting 문제라면 56-layer는 test error에서만 20-layer보다 높아야 할 것입니다. 그러나 training error도 높다는 점이 보이네요.
  
![figure1](https://user-images.githubusercontent.com/68625698/95048711-4d79a680-0723-11eb-8142-522951c0ccb8.PNG)
  
즉, 이 degradation 문제는 모든 시스템의 최적화 난이도가 비슷한 것이 아니라는 것을 보여줍니다. 그래서 얕은 architecture와 깊은 architecture를 비교해보려 합니다. 얕은 구조에 identity mapping을 진행하는 layer를 붙이는 방식을 진행했으나 이는 좋은 해결책이 아니라는 결론을 얻었습니다.  
  
그래서 이 논문에서는 __deep residual learning 프레임워크로 degradation 문제를 해결__했습니다. 새롭게 쌓는 layer를 underlying mapping 하는 것이 아니라, residual mapping을 하는 것입니다. 원래의 mapping이 __H(x)__라면 stacked nonlinear layer의 mapping은 __F(x) = H(x) - x__로 이를 학습하는 것입니다.즉, 원래의 mapping은 __H(x) = F(x) + x__가 되겠네요.  
  
저자들을 만약 identity mapping이 optimal하다면, __H(x)의 결과가 x로 만드는 것보다 F(x)를 0으로 학습하는 것이 쉬울 것__이라는 가정을 제시하며 실험을 진행합니다.  
공식 F(x) + x는 shortcut connections이라고 불리는데, 하나 이상의 layer를 넘어가는 것을 말합니다. 여기서는 shortcut connection을 identity mapping에서 진행하고, 그 결과를 stacked layer의 output에 더해주고 있습니다.  
  
![figure2](https://user-images.githubusercontent.com/68625698/95048712-4d79a680-0723-11eb-8f3d-77508760bb4b.PNG)
  
위의 그림을 보면 이해하기 수월합니다. x는 layer들을 통과하여 F(x)라는 값을 도출합니다. 그리고 나온 output에 x를 더함으로써 F(x)+x의 값을 도출해주고 있습니다. 이 때 x는 하나 이상의 레이어를 건너 뛰는 것을 볼 수 있습니다. 이를 shortcut connection이라고 합니다.  
  
이와 같은 __identity shortcut connection은 별도의 parameter나 computational complexity가 추가되지 않는다__는 장점을 지닙니다.  
  
    
논문은 실험을 통해 2가지 목표를 보여주려고 합니다.    
  
  
   __1. plain network는 depth가 깊어지면 training error가 높아지는 반면에, 제시한 deep residual network는 쉽게 최적화가 가능하다.__
     
   __2. deep residual network는 깊어진 depth에서 성능이 좋아져 이전의 연구들보다 더 좋은 결과를 얻을 수 있다.__
     
이를 바탕으로 실험한 결과, 이전의 network들보다 훨씬 뛰어난 결과를 얻을 수 있었다.

## Deep Residual Learning

### Residual Learning  
    
앞에서 이야기했던 내용이 반복됩니다. H(x)를 기존 mapping이라고 가정합니다. 이는 점차 복잡한 함수에 근사된다고 가정하면, F(x) = H(x) - x도 근사시킬 수 있다고 가정합니다. 또한 H(x)의 근사보다 F(x)의 근사가 훨씬 쉬우므로 이를 바탕으로 모델링을 진행하는 것입니다.  
  
  
### Identity Mapping by Shortcuts  
  
Shortcut connection을 공식으로 설명한 부분입니다. 위의 그림에서 보듯, H(x) = F(x) + x 입니다. 이를 더 자세히 나타네면 아래의 공식들과 같이 나옵니다.  
  
![figure3](https://user-images.githubusercontent.com/68625698/95048714-4e123d00-0723-11eb-83eb-ad22d64b6feb.PNG)
![figure4](https://user-images.githubusercontent.com/68625698/95048716-4e123d00-0723-11eb-8359-fbed2152b44e.PNG)  
  
1번 공식과 같이 도출할 수도 있으며, 이를 더 약식으로 나타내서 bias를 제거한 형태가 2번 공식입니다.  
  
1번 공식의 특징은 parameter나 comutation complexity가 증가하지 않는다는 점입니다. 이 점은 parameter, depth 등을 고정시키고 오로지 plain network와의 accuracy만을 비
교할 수 있으므로 굉장히 중요합니다.  
  
2번 공식은 x와 F의 차원이 다르면 사용하는 공식입니다. 이 때, $W_s$는 차원을 맞추는 용도로만 사용됩니다.(degradation 문제는 identity mapping으로 충분함) 아래의 그림을 보면 좀 더 쉽게 이해할 수 있습니다.
  
![figure5](https://user-images.githubusercontent.com/68625698/95048717-4eaad380-0723-11eb-9543-1279eefa5860.PNG) 
  
### Network Architectures  
  
우선 plain network와 residual network 2개에만 초점을 맞춥니다. network 구성은 아래와 같습니다.  
  
![figure6](https://user-images.githubusercontent.com/68625698/95048719-4f436a00-0723-11eb-8110-bce86b287536.PNG)
  
plain network는 VGG net을 바탕으로 진행했습니다. conv layer는 3 * 3의 필터를 가지며 2가지의 규칙으로 network는 디자인됐습니다.  
1. 동일한 output feature map size에 대해, layer는 동일한 수의 filter를 갖는다.  

2. feature map size가 절반인 경우, layer 당의 time complexity를 보전하기 위해 filter의 수는 2배로 한다.  

downsampling 시에는 stride가 2인 conv layer를 사용하며 마지막엔 global average pooling layer를 사용하며 activation이 softmax를 사용합니다. 이 architecture 자체는 VGG보다 낮은 복잡도와 적은 필터를 가지고 있습니다.  
  
Residaul Network의 경우, 앞서 만든 plain network를 기반으로 shortcut connection을 삽입해 만듭니다. 만약 input과 output의 차원이 다를 경우, 2가지 중 하나를 선택하여 적용합니다.
1. zero entry를 추가로 padding하여 dimension을 맞춘 후, identity mapping을 진행한다.    

2. projection shortcut을 dimension을 맞출 때 사용한다.(위의 2번 공식을 사용한다는 것입니다.)  
  
두 옵션 중 어떤 것을 선택하든 strides는 2로 고정입니다.  
 
### Implementation  
  
위의 내용들은 실험을 위해 설계된 network들입니다. 실험을 진행하고 그 결과를 바탕으로 모델을 짜고 ImageNet Challenge에 나갔습니다. 이 때, 아래와 같은 점들을 반영하여 모델을 수정하고 challenge에 나간 것입니다. (읽고 넘어가도 무방합니다.)  
  
    weight decay 0.00001 & momentum 0.9
    drop out X
    image는 [256, 480] 중 가까운 방향으로 resize
    per_pixel mean subtraction과 함께 224 * 224로 랜덤 crop
    standard color augmentation 사용
    batch normalization을 convolution 후, activation 전 사용
    batch size는 256이며 SGD 사용
    learning rate는 0.1부터 시작하여 local minimum을 만나거나 loss가 진동하면 1/10씩 감소
    iteration은 60 * 10^4

## Experiments  

### ImageNet Classification

논문에서는 ImageNet classification dataset으로 학습과 평가를 진행했습니다. 평가는 top-1, top-5 error를 사용했습니다.  
  
우선 plain network부터 보겠습니다. 18-layer와 34-layer 모두를 확인했습니다. 그 결과 오히려 34 layer가 validation error가 높았습니다.  

![figure7](https://user-images.githubusercontent.com/68625698/95048720-4f436a00-0723-11eb-9dbc-0e9d4f6b7bae.PNG)

혹자는 이런 현상이 발생하는 이유가 vanishing gradients때문이라고 생각할 수 있으나 그 때문은 아닙니다. 논문에선 batch normalization을 통해 이를 미리 방지했으며, backward propagated gradients 역시 괜찮은 것을 확인했기 때문입니다. 오히려 저자들은 exponentially low convergence rate가 training error의 감소에 영향을 끼쳤을 것이라고 생각합니다.  
  
Residual Networks도 plain networks를 바탕으로 똑같이 18-layer와 34-layer 구조로 모델을 만들었습니다. 다른 점은 shortcut connection을 추가한 것 입니다. shortcut connection은 identity mapping을 이용하며, 차원이 다를 경우, zero padding을 쓰기로 했습니다. 따라서 plain networks에 비해 추가되는 parameter들은 없습니다.  

여기서 중요한 점은 3가지 입니다.  
1. 34 layer일 때, 성능이 개선됐다. layer의 깊이가 깊어질 수록 높은 accuracy를 얻었다. 이는 gradation problem을 해결한 것이다.

2. 34-layer는 plain networks와 비교할 때, top-1 error를 3.5% 정도 줄였다.

3. ResNet은 빠른 수렴속도를 기대할 수 있다.  
  
### Identity vs Projection Shortcuts.  
  
앞선 내용에서는 parameter - free를 바탕으로 논문을 진행했습니다. 이번에는 projection shortcut에 대해 알아봅시다. 아래의 3가지 옵션을 바탕으로 비교하려고 합니다.

 - A : 차원을 맞추기 위해 zero padding을 사용
 - B : projection shortcut이 차원을 늘릴 때만 사용
 - C : 모든 shortcut은 projection인 경우  
   
기본적으로 세 가지 옵션 모두 plain network 보다는 성능이 우수하다. 옵션 간에 비교를 하자면 A < B < C 순이었다. 이 이유는 A의 경우, zero padding으로 추가된 차원은 residual learning을 제대로 못 쓰기 때문이다. B의 경우, projection shortcut에 의해 추가된 parameter가 성능에 영향을 줬을 것으로 보인다.  

그러나 여기서 알아야 할 점은 세 가지의 성능이 큰 차이가 없다는 것이다. 즉, projection shortcut이 필수적이지는 않다는 것이다. 그래서 논문에서는 굳이 C를 해보지 않는다.
  
### Deeper Bottleneck Architectures  
  
위의 모델들을 그냥 사용하게 되면 실제 사용 시, training time은 걷잡을 수 없게 된다. 이를 개선하기 위해 bottleneck design으로 바꿨다. 2 layer를 3 layer로 바꾸며 구조는 1 * 1, 3 * 3, 1 * 1의 conv layer 형태이다. 1 * 1은 차원을 줄이거나 늘리는 용도로 사용하고, 3 * 3을 bottleneck으로 사용한다. 아래의 그림을 보면 더 쉽게 이해할 수 있다.  

![figure8](https://user-images.githubusercontent.com/68625698/95048722-4fdc0080-0723-11eb-8716-b14afb86f9ec.PNG)
  
이 때 중요한 것은 identity shortcut이어야 한다는 점이다. projection으로 대체되면 복잡성은 2배로 늘어나게 된다.  
  
- 50 layer ResNet : 34 layer ResNet에 3 layer bottleneck block으로 대체하여 구현했다. 옵션 b를 사용했고, 모델은 3.8 billion FLOPs이다.  
  
- 101 layer and 152 layer ResNets : 50 layer ResNet에 3 layer bottleneck block을 추가해 101 layer와 152 layer를 만들었다. depth가 굉장히 깊어졌음에도 불구하고 굉장히 높은 정확도를 도출했다.  
  
아래의 표를 보면 확실히 비교할 수 있다.

![figure9](https://user-images.githubusercontent.com/68625698/95048723-4fdc0080-0723-11eb-9313-85b43eac31c7.PNG)
  

### Comparisons with State-of-the-art Methods.  

저자들은 6개의 depth가 다른 모델들을 앙상블하여 top-5 test error를 3.57%까지 달성했으며 ILSVRC 2015 classification task에서 1위를 했습니다.  
 
 
## CIFAR-10 and Analysis  
  
ResNet을 가지고 CIFAR-10 dataset을 적용해봤다는 내용입니다. 전반적인 내용 진행이나 성과는 앞선 내용과 비슷하므로 생략하겠습니다. 여기서 눈에 띄는 것은 1000개 이상의 layer를 쌓아봤다는 것입니다. 그 결과 110 layer ResNet보다 testing result가 낮게 나왔는데 이는 overfitting으로 추측됩니다. 실제로 모델에 maxout이나 dropout은 진행되지 않았다고 합니다. 이를 모델에 적용하면 더 나은 결과를 얻을 것으로 예상됩니다.

# Summary  
  - 이 부분은 수식이나 증명이 없는 keyword를 바탕으로 요약된 부분입니다. 자세한 흐름은 윗 부분을 읽어주세요 :)
  
### Degradation Problem  

network의 depth가 깊어지면 깊어질수록, 성능이 떨어지는 문제를 말합니다. 상식적으로 생각했을 때, depth가 깊어지면 더 많은 feature들을 찾아낼 것이고, 찾아낸 feature들을 바탕으로 test를 하므로 성능이 좋아질 것이라고 생각합니다. 하지만 ResNet 이전의 모델들은 그러질 못하고 위의 문제를 대면했습니다. 그래서 모델을 만들 때, 최적화된 layer의 depth를 설정해둔 것입니다. 하지만 ResNet은 이 문제를 Residual Learning, Skip Connection, Identity mapping을 바탕으로 극복했습니다.  
  
### Identity mapping  

ResNet의 시작은 Identity mapping부터 였습니다. 저자들은 '지금 모델의 layer로부터 도출되는 결과가 최적이라 가정하자. 그렇다면 현재 모델에 layer를 더 쌓고 그 결과값이 그대로 나오게 한다면 최소한 성능이 동일하거나 개선되지 않을까?'라는 생각을 갖게 됩니다. 그래서 모델에 Identity mappaing을 하는 layer를 쌓은 뒤, 실험해봤으나 결과는 예상과는 달랐습니다.  
  
### Skip Connection

위 실험 이후, 저자들은 생각을 바꿨습니다. 단순히 H(x) = x 인 것이 아니라 함수를 통과하는 것이므로 H(x) = F(x) + x 일 것이라고 말이죠. 즉, 통계적인 관점에서 식을 세우고 skip connection을 사용한 것입니다.(구체적인 내용은 위에서 서술해뒀음) 그리고 만약 H(x)를 실제값과 근사시킬 수 있다면, F(x) 역시 0로 근사시킬 수 있는 것이며, 위와 같은 방법이 더 쉽다고 서술합니다. 어떻게 보면 넓은 변동을 잡던 기존의 식을 상대적으로 작은 변동을 잡는다는 의미로 볼 수도 있을 것 같습니다. 이 같은 과정을 가능케 하기 위해서 skip connection을 적용한 것인데요. x를 함수 F()에 넣음으로써 F(x) 값을 구함과 동시에 F를 skip해 결과값 F(x)에 x를 더해주는 것입니다.

### Residual Learning

위와 같은 방법을 Residual Learning이라고 합니다. 즉, layer가 기존의 mappaing 방식을 학습하게 하는 것이 아닌 잔차인 F(x)를 학습케 하는 것이죠. 실제로 논문에서는 실험을 통해 이를 증명해냈습니다. 실험 결과, 모델의 깊이는 깊어지고 성능은 높아졌습니다. 또한 모델에 사용되는 parameter나 computataion complexity 등은 훨씬 줄어드는 모습을 보였습니다. 



# Reference  
https://sike6054.github.io/blog/paper/first-post/  
https://curaai00.tistory.com/1  
9주차 모델심화(CNN) 강의

# 아래의 사진은 과제1 CRNN 모델 구현 후 train에 성공한 화면 입니다. 총 30회 중 최종 모델을 저장한 부분을 캡쳐했습니다.  
  
![assignment1](https://user-images.githubusercontent.com/68625698/95048708-4bafe300-0723-11eb-970b-0c00321c1b03.png)